In [16]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
import sys
sys.path.append("../src")
from myproject.pipelines.clean_pipeline import cleaning_pipeline
from myproject.pipelines.preprocessing_pipeline import preprocessing_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    accuracy_score
)
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform


In [ ]:
df_raw = pd.read_csv('/home/jakub/Pulpit/Credit original/data/raw/train.csv')
df = df_raw

/tmp/ipykernel_8072/2618218542.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('/home/jakub/Pulpit/Credit original/data/raw/train.csv')


In [18]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


Sekcja 1

In [19]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [34]:
X.shape

(100000, 27)

In [36]:
y.shape

(100000,)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
cleaning_pipeline1 = cleaning_pipeline()
preprocessing_pipeline1 = preprocessing_pipeline()

clean_pipeline = cleaning_pipeline()


In [22]:
log_reg_clf = Pipeline([
    ('cleaner', cleaning_pipeline1),
    ('preprocess', preprocessing_pipeline1),
    ('model', LogisticRegression(random_state=42))
])

In [23]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = ['accuracy', 'f1_macro']

crossv_results = cross_validate(
    log_reg_clf,
    X_train,
    y_train,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False
)

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline

In [24]:
#Cross Validacaj wyniki

#Średnie
cv_acc_mean = crossv_results['test_accuracy'].mean()
cv_f1_mean = crossv_results['test_f1_macro'].mean()

#Odchylenie
cv_acc_std = crossv_results['test_accuracy'].std()
cv_f1_std = crossv_results['test_f1_macro'].std()

In [25]:
#Cross Validacja wyniki:

print(f'CV accuracy mean: {cv_acc_mean:.6f}')

print(f'CV f1 mean: {cv_f1_mean:.6f}')

print(f'CV accuracy std: {cv_acc_std:.6f}')

print(f'CV f1 sdt {cv_f1_std:.6f}')

CV accuracy mean: 0.630825
CV f1 mean: 0.581423
CV accuracy std: 0.002326
CV f1 sdt 0.003962


In [26]:
log_reg_clf.fit(X_train, y_train)

,steps,"[('cleaner', ...), ('preprocess', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('drop', ...), ('num_clean', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,target_cols,"['ID', 'Customer_ID', ...]"
,chars_to_remove,"'[^0-9.,]+'"
,target_cols,"['Age', 'Annual_Income', ...]"


In [27]:
y_pred = log_reg_clf.predict(X_test)

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [28]:

print("=== Classification report (test) ===")
print(classification_report(y_test,y_pred))

=== Classification report (test) ===
              precision    recall  f1-score   support

        Good       0.55      0.40      0.47      3566
        Poor       0.66      0.50      0.57      5799
    Standard       0.63      0.78      0.70     10635

    accuracy                           0.63     20000
   macro avg       0.62      0.56      0.58     20000
weighted avg       0.63      0.63      0.62     20000



In [29]:
f1_test = f1_score(y_test, y_pred, average="macro")
print(f"\nF1-macro (test): {f1_test:.4f}")


F1-macro (test): 0.5775


Sekcja 2

In [30]:
# param_distributions_= {
#     "model__C": loguniform(1e-3,1e3),
#         "model__penalty": ["l2"],
#         "model__solver":["lbfgs"],
#         "model__class_weight": [None, "balanced"],
#         "model__multi_class": ["multinomial"],
#         "model__max_iter":[300,500,1000],
# }

# random_search = RandomizedSearchCV(
#     estimator=log_reg_clf,
#     param_distributions=param_distributions_,
#     n_iter=30,
#     scoring="f1_macro",
#     cv=cv,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42,
# )

# random_search.fit(X_train, y_train)

# print("Najlepsze parametry: ", random_search.best_params_)
# print('Najlepsze f1_macro: ', random_search.best_score_)

In [31]:
# best_logreg = random_search.best_estimator_

# y_pred_tuned = best_logreg.predict(X_test)

# f1_test = f1_score(y_test, y_pred_tuned, average='macro')
# acc_test = accuracy_score(y_test, y_pred_tuned)

# print("F1-macro (TEST, tuned): ", f1_test)
# print("Accuracy (TEST, tuned): ", acc_test)
# print(classification_report(y_test, y_pred_tuned))
# cm= confusion_matrix(y_test, y_pred_tuned)
# cm

In [32]:
# import mlflow
# mlflow.set_experiment("First Experiment")
# mlflow.set_tracking_uri("http://127.0.0.1:5000")

# with mlflow.start_run(run_name="Logistic Regresion Tuned"):
#     mlflow.log_params(random_search.best_params_)
#     mlflow.log_metrics({
#         #metryki z CV (z tuningiem)
#         mlflow.log_metrics("cv_f1_macro_mean", random_search.best_score_)
#         #metryki z testu
        
#         mlflow.log_metrics("test_f1_")
#     })
    
#     mlflow.sklearn.log_model(log_reg_clf, 'Logistic Regression no params')